In [1]:
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from torch import save 
import copy

### 数据预处理

In [41]:
# train_df = pd.read_csv("./data/train.csv")
# train_df = pd.read_csv("./data/train.csv").loc[0:800]
train_df = pd.read_csv("./data/train.csv")
train_df = train_df[["PassengerId", "Survived", "Pclass", "Sex", "Age", "SibSp", "Embarked", "Parch", "Fare"]]  
for index, x in train_df.iterrows():
    x["SibSp"] = "s" + str(x["SibSp"])

train_df = train_df.fillna({"Embarked":"Q", "Age": 0})
train_df = pd.get_dummies(train_df) # 独热编码，也可以二值化
Pclass_1,Pclass_2,Pclass_3 = [],[],[]

for index, line in train_df.iterrows():
    if line["Pclass"] == 1.0:
        Pclass_1.append(1)
        Pclass_2.append(0)
        Pclass_3.append(0)
    elif line["Pclass"] == 2.0:
        Pclass_1.append(0)
        Pclass_2.append(1)
        Pclass_3.append(0)
    else:
        Pclass_1.append(0)
        Pclass_2.append(0)
        Pclass_3.append(1)
    
        
train_df["Pclass_1"], train_df["Pclass_2"], train_df["Pclass_3"] = Pclass_1, Pclass_2, Pclass_3


# train_df['SibSp_1'] = np.array(train_df['SibSp'] == 1).astype(np.int32)
# train_df['SibSp_0'] = np.array(train_df['SibSp'] == 0).astype(np.int32)
# train_df['SibSp_2'] = np.array(train_df['SibSp'] == 2).astype(np.int32)
# train_df['SibSp_3'] = np.array(train_df['SibSp'] == 3).astype(np.int32)
# train_df['SibSp_4'] = np.array(train_df['SibSp'] == 4).astype(np.int32)
# train_df['SibSp_6'] = np.array(train_df['SibSp'] == 6).astype(np.int32)
# train_df['SibSp_8'] = np.array(train_df['SibSp'] == 8).astype(np.int32)
# train_df.drop("SibSp", axis=1)


# test_df = pd.read_csv("./data/train.csv").loc[800:]
test_df = pd.read_csv("./data/test.csv")
test_df  = test_df[["PassengerId", "Pclass", "Sex", "Age", "SibSp", "Embarked", "Parch", "Fare"]]
test_df = test_df.fillna({"Embarked":"C", "Age": 0})
test_df = pd.get_dummies(test_df)
Pclass_1,Pclass_2,Pclass_3 = [],[],[]
for index, line in test_df.iterrows():
    if line["Pclass"] == 1.0:
        Pclass_1.append(1)
        Pclass_2.append(0)
        Pclass_3.append(0)
    elif line["Pclass"] == 2.0:
        Pclass_1.append(0)
        Pclass_2.append(1)
        Pclass_3.append(0)
    else:
        Pclass_1.append(0)
        Pclass_2.append(0)
        Pclass_3.append(1)
test_df["Pclass_1"], test_df["Pclass_2"], test_df["Pclass_3"] = Pclass_1, Pclass_2, Pclass_3


# test_df['SibSp_1'] = np.array(test_df['SibSp'] == 1).astype(np.int32)
# test_df['SibSp_0'] = np.array(test_df['SibSp'] == 0).astype(np.int32)
# test_df['SibSp_2'] = np.array(test_df['SibSp'] == 2).astype(np.int32)
# test_df['SibSp_3'] = np.array(test_df['SibSp'] == 3).astype(np.int32)
# test_df['SibSp_4'] = np.array(test_df['SibSp'] == 4).astype(np.int32)
# test_df['SibSp_6'] = np.array(test_df['SibSp'] == 6).astype(np.int32)
# test_df['SibSp_8'] = np.array(test_df['SibSp'] == 8).astype(np.int32)
# test_df.drop("SibSp", axis=1)


labels = np.array(train_df['Survived'], dtype=np.float32)
#labels = np.array(train_df['Survived'], dtype=np.float32)
new_labels = []
for i in labels:
    # 死亡 活着
    #  1    0
    #  0    1
    
    if i==1: 
        new_labels.append([0, 1])
    else:
        new_labels.append([1, 0])
labels = np.array(new_labels)
print(labels)
# train_df = train_df.drop(columns=["Survived", "Pclass","SibSp",],) 
train_df = train_df.drop(columns=["Survived", "Pclass",],) 
# test_df = test_df.drop(columns=["Pclass","SibSp",],)
test_df = test_df.drop(columns=["Pclass",],)
train_features = np.array(StandardScaler().fit_transform(train_df), dtype=np.float32)
test_features = np.array(StandardScaler().fit_transform(test_df), dtype=np.float32)
# test_features = np.array(test_df, dtype=np.float32)
# train_features = np.array(train_df, dtype=np.float32)
train_df.shape, test_df.shape

[[1 0]
 [0 1]
 [0 1]
 ...
 [1 0]
 [0 1]
 [1 0]]


((891, 13), (418, 13))

### 定义网络模型

In [42]:
from torch.nn import functional as F
input_size = train_features.shape[1]

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 7)
#         self.hidden1 = nn.Linear(8, 4)
#         self.hidden2 = nn.Linear(256, 80)
        self.out = nn.Linear(7, 2)
    
    def forward(self, x):
        x = F.relu(self.linear(x))
#         x = F.relu(self.hidden1(x))
#         x = F.relu(self.hidden2(x))
       # x = F.relu(self.hidden3(x))
        out = self.out(x)
        return out
    
net = LinearRegressionModel()   
criterion = nn.MSELoss() 

### 训练模型

In [45]:
batch_size = 12
num_epochs = 1000
train_features = torch.tensor(train_features, dtype=torch.float)
labels = torch.tensor(labels, dtype=torch.float)
train_ds = TensorDataset(train_features, labels)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True) # MINI-Batch

c:\users\jerry\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\jerry\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [46]:


optimizer = optim.Adam(net.parameters(), lr=1e-4)
best_model_wts = net.state_dict()
best_acc = 0



for epoch in range(num_epochs):
    net.train()
    
    total = 0
    corrects = 0
    for x_batch, y_batch in train_dl:
        total +=1
        optimizer.zero_grad()
        #print(x_batch.shape[1])
        outputs = net(x_batch)
        answer_ls = list()
        for a, b in outputs:
            if a>b:
                answer_ls.append([1, 0])
            else:
                answer_ls.append([0, 1])
        if torch.tensor(answer_ls, dtype=torch.float).equal(y_batch):
            corrects += 1
        
        loss = criterion(outputs, y_batch)
        

        loss.backward(retain_graph=True)
        optimizer.step()
    
    acc = corrects/total*100
    if acc > best_acc:
        best_model_wts = copy.deepcopy(net.state_dict())
        best_acc = acc
    if epoch%25==0:
        print("epoch:{} loss:{} correct:{} acc:{}%".format(epoch, loss.item(), corrects, corrects/total*100))

epoch:0 loss:0.03405700996518135 correct:8 acc:10.666666666666668%
epoch:25 loss:0.07722539454698563 correct:6 acc:8.0%
epoch:50 loss:0.26206961274147034 correct:9 acc:12.0%
epoch:75 loss:0.12442474812269211 correct:8 acc:10.666666666666668%
epoch:100 loss:0.010081815533339977 correct:7 acc:9.333333333333334%
epoch:125 loss:0.4681619703769684 correct:6 acc:8.0%
epoch:150 loss:0.023560017347335815 correct:5 acc:6.666666666666667%
epoch:175 loss:0.004121491685509682 correct:10 acc:13.333333333333334%
epoch:200 loss:0.05554385110735893 correct:8 acc:10.666666666666668%
epoch:225 loss:0.07921872287988663 correct:5 acc:6.666666666666667%
epoch:250 loss:0.25026562809944153 correct:9 acc:12.0%
epoch:275 loss:0.14298667013645172 correct:9 acc:12.0%
epoch:300 loss:0.005576417315751314 correct:9 acc:12.0%
epoch:325 loss:0.10616409778594971 correct:8 acc:10.666666666666668%
epoch:350 loss:0.2021588832139969 correct:4 acc:5.333333333333334%
epoch:375 loss:0.013818356208503246 correct:14 acc:18.666

In [49]:

#print("test features", test_features)
net.load_state_dict(best_model_wts)
train_arr = np.array(train_features, dtype=np.float)
tensors = torch.tensor(train_arr, dtype=torch.float32)

outputs = net(tensors).data.numpy()
ans = []
for a, b in outputs:
    if a>b:
        ans.append(1)
    else:
        ans.append(0)
answer = np.array(ans).tolist()

        
lab = []
labels = np.array(labels).tolist()
for a, b in labels:
    if a==1:
        
        lab.append(1)
    else:
        lab.append(0)        
length = len(ans)

correct = 0

print(lab)
print(ans)
for a in range(length):
    if lab[a] == answer[a]:
        correct+=1

correct/length*100


[1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 

c:\users\jerry\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


83.27721661054994

In [47]:
test_data = torch.from_numpy(test_features)
predict = net(test_data)
outputs = predict.data.numpy()
answer_ls = []
for a, b in outputs:
    if a>b:
        answer_ls.append([1, 0])
    else:
        answer_ls.append([0, 1])
lab = []
for a, b in answer_ls:
    if a==1:
        lab.append(0)
    else:
        lab.append(1)          
outputs = np.array(lab)


In [48]:
dataframe = pd.DataFrame({
    "PassengerId": np.array(range(892, 1310)),
    "Survived": outputs
})
dataframe.to_csv("./results.csv")

[1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 

c:\users\jerry\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


'0.0'

In [51]:
1e-3

0.001